In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals


import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_pickle("dataframes/full_df_cleaned_train_0_8.pickle")

In [11]:
df_test = pd.read_pickle("dataframes/full_df_cleaned_test_0_2.pickle")

In [12]:
count_unique_words(df)

452521

In [24]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    for row in range(embedding_matrix.shape[0]):
        if not np.any(embedding_matrix[row,:]):
            embedding_matrix[row,:] = np.random.rand(embedding_dim)

    return embedding_matrix

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.sentence)
X_train = tokenizer.texts_to_sequences(df.sentence)
X_test = tokenizer.texts_to_sequences(df_test.sentence)

#def max_len(X):
#    maxlen = 0
#    for el in X:
#        maxlen = maxlen if len(el) < maxlen else len(el)
#    return maxlen
#maxlen = max_len(X_train)
maxlen=16

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

y_train = np.where(df.label == 1, 1, 0)
y_test = np.where(df_test.label == 1, 1, 0)

In [15]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

396986

In [16]:
df.iloc[8634]

sentence    want pottermore let ! can't obsessed let #nerd...
label                                                      -1
Name: 2109505, dtype: object

In [17]:
print(X_train[0, :])

[  15 1810 8634 2884 3383    7    0    0    0    0    0    0    0    0
    0    0]


In [18]:
embedding_dim = 50

In [25]:
embedding_matrix = create_embedding_matrix(
    'glove/glove.twitter.27B.50d.txt',
    tokenizer.word_index, embedding_dim)

In [20]:
print(X_train.shape)

(2000000, 16)


In [21]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(nonzero_elements / vocab_size)
len(embedding_matrix)

0.3841621618898399


396986

In [21]:
#without embedding matrix
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 44, 50)            19849300  
_________________________________________________________________
conv1d (Conv1D)              (None, 40, 128)           32128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 19,882,729
Trainable params: 19,882,729
Non-trainable params: 0
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath="models/cnn"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    batch_size=512)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)


Train on 2000000 samples, validate on 500000 samples
Epoch 1/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3986 - accuracy: 0.8125WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2019-12-17 03:15:52,911 : WARNING : Can save best model only with val_acc available, skipping.


2000000/2000000 [==============================] - 3115s 2ms/sample - loss: 0.3986 - accuracy: 0.8125 - val_loss: 0.3729 - val_accuracy: 0.8269
Epoch 2/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3257 - accuracy: 0.8536WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2019-12-17 04:12:35,618 : WARNING : Can save best model only with val_acc available, skipping.


2000000/2000000 [==============================] - 3403s 2ms/sample - loss: 0.3257 - accuracy: 0.8536 - val_loss: 0.3736 - val_accuracy: 0.8275
Epoch 3/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.2721 - accuracy: 0.8811WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

2000000/2000000 [==============================] - 3633s 2ms/sample - loss: 0.1366 - accuracy: 0.9400 - val_loss: 0.8776 - val_accuracy: 0.8043
Epoch 14/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.1329 - accuracy: 0.9413WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2019-12-17 14:48:28,037 : WARNING : Can save best model only with val_acc available, skipping.


2000000/2000000 [==============================] - 3737s 2ms/sample - loss: 0.1329 - accuracy: 0.9413 - val_loss: 0.9405 - val_accuracy: 0.8034
Epoch 15/50
 765440/2000000 [==========>...................] - ETA: 30:50 - loss: 0.1233 - accuracy: 0.9458WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2019-12-17 15:07:35,267 : WARNING : Can save best model only with val_acc available, skipping.


KeyboardInterrupt: 

In [34]:
#with embedding matrix
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 44, 50)            19849300  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 40, 128)           32128     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 19,882,729
Trainable params: 19,882,729
Non-trainable params: 0
_________________________________________________________________


In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath="models/cnn"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    batch_size=512)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

Train on 2000000 samples, validate on 500000 samples
Epoch 1/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.4064 - accuracy: 0.8068
Epoch 00001: val_accuracy improved from -inf to 0.82180, saving model to models/cnn
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2019-12-17 18:25:13,768 : WARNING : From /Users/cmujdei/opt/anaconda3/envs/twits/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: models/cnn/assets


2019-12-17 18:25:16,971 : INFO : Assets written to: models/cnn/assets


2000000/2000000 [==============================] - 6753s 3ms/sample - loss: 0.4064 - accuracy: 0.8068 - val_loss: 0.3822 - val_accuracy: 0.8218
Epoch 2/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3535 - accuracy: 0.8382
Epoch 00002: val_accuracy improved from 0.82180 to 0.82544, saving model to models/cnn
INFO:tensorflow:Assets written to: models/cnn/assets


2019-12-17 19:25:43,319 : INFO : Assets written to: models/cnn/assets


2000000/2000000 [==============================] - 3626s 2ms/sample - loss: 0.3535 - accuracy: 0.8382 - val_loss: 0.3756 - val_accuracy: 0.8254
Epoch 3/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3222 - accuracy: 0.8550
Epoch 00003: val_accuracy did not improve from 0.82544
2000000/2000000 [==============================] - 3345s 2ms/sample - loss: 0.3222 - accuracy: 0.8550 - val_loss: 0.3836 - val_accuracy: 0.8251
Epoch 4/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.2980 - accuracy: 0.8668
Epoch 00004: val_accuracy did not improve from 0.82544
2000000/2000000 [==============================] - 3163s 2ms/sample - loss: 0.2980 - accuracy: 0.8668 - val_loss: 0.3977 - val_accuracy: 0.8233
Epoch 5/50
  91648/2000000 [>.............................] - ETA: 48:55 - loss: 0.2667 - accuracy: 0.8833WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


2019-12-17 21:16:31,524 : WARNING : Can save best model only with val_accuracy available, skipping.


KeyboardInterrupt: 

In [24]:
#using Stefano's link
from keras.layers import Embedding

embedding_layer = Embedding(vocab_size, 
                            embedding_dim, 
                            weights=[embedding_matrix], 
                            input_length=maxlen, 
                            trainable=False)

Using TensorFlow backend.


In [32]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.MaxPooling1D(35))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

ValueError: Negative dimension size caused by subtracting 5 from 4 for 'max_pooling1d_1_1/MaxPool' (op: 'MaxPool') with input shapes: [?,4,1,128].

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath="models/cnn"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    batch_size=512)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

In [29]:
#glove twitter

from keras.layers import GlobalMaxPooling1D, concatenate, Dropout, Dense, Embedding, Input, Conv1D
from keras.models import Model

# Specifying the input shape: the input is a sentence of maxlen words
embedding_layer = Embedding(vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen)
sequence_input = Input(shape=(maxlen,), dtype='int32')
# Creating the embedding using the previously constructed embedding matrix
embedded_sequences = embedding_layer(sequence_input)
convs = []
filter_sizes = [3,5]
for filter_size in filter_sizes:
    # Creating the convolutional layer:
    #    "filters" represents the number of different windows we want (i.e. how many channels to produce),
    #    therefore in our case we will end up with 200 different convolutions
    conv_layer = Conv1D(filters=100, 
                    kernel_size=filter_size, 
                    activation='relu')(embedded_sequences)
    # Creating the global max pooling layer
    pool_layer = GlobalMaxPooling1D()(conv_layer)
    convs.append(pool_layer)
merged_layers = concatenate(convs, axis=1)
# Create dropout leayer: randomly set a fraction of input units to 0, which helps prevent overfitting
x = Dropout(0.3)(merged_layers)  
# Create (regular) densely-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation='sigmoid')(x)
model_tw = Model(sequence_input, preds)
model_tw.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_tw.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 16, 50)       19849300    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 14, 100)      15100       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 12, 100)      25100       embedding_3[0][0]                
____________________________________________________________________________________________

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath="models/cnn_glove_tw"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model_tw.fit(X_train, y_train, epochs=50, verbose=True, validation_data=(X_test, y_test), callbacks=callbacks_list, batch_size=512)
loss, accuracy = model_tw.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_tw.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

/Users/cmujdei/opt/anaconda3/envs/twits/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000000 samples, validate on 500000 samples
Epoch 1/50
2000000/2000000 [==============================] - 3680s 2ms/step - loss: 0.4118 - accuracy: 0.8044 - val_loss: 0.3797 - val_accuracy: 0.8235

Epoch 00001: val_accuracy improved from -inf to 0.82353, saving model to models/cnn_glove_tw
Epoch 2/50
2000000/2000000 [==============================] - 6786s 3ms/step - loss: 0.3664 - accuracy: 0.8316 - val_loss: 0.3686 - val_accuracy: 0.8288

Epoch 00002: val_accuracy improved from 0.82353 to 0.82879, saving model to models/cnn_glove_tw
Epoch 3/50
2000000/2000000 [==============================] - 4198s 2ms/step - loss: 0.3389 - accuracy: 0.8466 - val_loss: 0.3698 - val_accuracy: 0.8298

Epoch 00003: val_accuracy improved from 0.82879 to 0.82983, saving model to models/cnn_glove_tw
Epoch 4/50
2000000/2000000 [==============================] - 5005s 3ms/step - loss: 0.3156 - accuracy: 0.8577 - val_loss: 0.3782 - val_accuracy: 0.8296

Epoch 00004: val_accuracy did not improve from